In [1]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

In [2]:
def create_df_from_1size(size1):

    df1 = pd.DataFrame(columns=['trap_x', 'trap_y', 'trap_z', 'x_init', 'y_init', 'z_init'])

    for i in range(360):
        path = f'rotation/Circle_{size1}/data360/circle_{size1}_{i}_degrees.csv'
        simulated = pd.read_csv(path)

        initial_x = simulated.loc[0, 'x']
        initial_y = simulated.loc[0, 'y']
        initial_z = simulated.loc[0, 'z']

        traps_path = f'rotation/Circle_{size1}/data360/traps_{size1}_{i}_degrees.csv'
        traps = pd.read_csv(traps_path)
        traps = traps.rename(columns={'x': 'trap_x', 'y': 'trap_y', 'z': 'trap_z'})
        traps['trap_x'] = traps['trap_x'] / 1000
        traps['trap_y'] = traps['trap_y'] / 1000
        traps['trap_z'] = traps['trap_z'] / 1000
        traps['x_init'] = initial_x 
        traps['y_init'] = initial_y
        traps['z_init'] = initial_z
        df1 = pd.concat([df1, traps])
    df1 = df1.reset_index()
    # df.drop(['index'])
    df1 = df1.drop('index', axis=1)
    return df1
    

In [3]:
df1 = create_df_from_1size(0.035)

In [ ]:
df2 = create_df_from_1size(0.07)

In [ ]:
df3 = create_df_from_1size(0.105)
df4 = create_df_from_1size(0.14)
df5 = create_df_from_1size(0.175)


In [ ]:
df = pd.DataFrame(columns=['trap_x', 'trap_y', 'trap_z', 'x_init', 'y_init', 'z_init'])



In [ ]:
df = df.append(df1[:1000])

In [ ]:
df

In [ ]:
for i in range(360):
    df = df.append(df1[i*1000:i*1000+1000])
    df = df.append(df2[i*1000:i*1000+1000])
    df = df.append(df3[i*1000:i*1000+1000])
    df = df.append(df4[i*1000:i*1000+1000])
    df = df.append(df5[i*1000:i*1000+1000])

In [ ]:
df

## RNN

In [ ]:
def get_train_test(df, split_percent=0.8):
    data = np.array(df.values.astype('float32'))
    n = len(data)
    # Point for splitting data into train and test
    split = int(n*split_percent)
    train_data = data[range(split)]
    test_data = data[split:]
    return train_data, test_data, data

In [ ]:
train, test, data = get_train_test(df)

In [ ]:
def get_XY(dat, time_steps):
    Y_ind = np.arange(time_steps, len(dat), time_steps)
    Y = dat[Y_ind]
    print(Y_ind)
    rows_x = len(Y)
    X = dat[range(time_steps*rows_x)]
    X = np.reshape(X, (rows_x, time_steps, 6)) 

    return X, Y

In [ ]:
time_steps = 25

trainX, trainY = get_XY(train, time_steps)
testX, testY = get_XY(test, time_steps)



In [ ]:
trainY = trainY[:, :3]
testY = testY[:, :3]

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [ ]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(SimpleRNN(hidden_units, input_shape=input_shape, activation=activation[0]))
#     model.add(Dense(units=10, activation=activation[1]))
    model.add(Dense(units=3, activation=activation[1]))


    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [ ]:
from time import perf_counter, sleep



In [ ]:
# Create model and train
model = create_RNN(hidden_units=3, dense_units=3, input_shape=(time_steps,6), 
                   activation=['tanh', 'tanh'])
start = perf_counter()
model.fit(trainX, trainY, epochs=5, batch_size=10, verbose=2, validation_split=0.1)
end = perf_counter()
print(f"Time taken to execute code : {end-start}")

In [ ]:
train_predict = model.predict(trainX)
test_predict = model.predict(testX)

In [ ]:
def print_error(trainY, testY, train_predict, test_predict):    
    # Error of predictions
    train_rmse = math.sqrt(mean_squared_error(trainY, train_predict))
    test_rmse = math.sqrt(mean_squared_error(testY, test_predict))
    # Print RMSE
    print('Train RMSE: %.9f RMSE' % (train_rmse))
    print('Test RMSE: %.9f RMSE' % (test_rmse)) 

In [ ]:
print_error(trainY, testY, train_predict, test_predict)

## Visualization of prediction 

In [ ]:
prediction_df = pd.DataFrame(columns=['x', 'y', 'z', 'label'])



In [ ]:
row = 2000

In [ ]:
prediction_df = pd.DataFrame(data = testX[row, :, :3], 
                  columns = ['x', 'y', 'z'])

In [ ]:
prediction_df['label'] = 'real_value'

In [ ]:
new_row = {'x':testY[row][0], 'y':testY[row][1], 'z':testY[row][2], 'label': 'prediction'}
prediction_df = prediction_df.append(new_row, ignore_index=True)

In [ ]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'notebook'
fig = px.line_3d(prediction_df, x="x", y="y", z="z", color='label',markers=True)
fig.show()